#### Instalasi Tensorflow Object Detection API

In [ ]:
!git clone https://github.com/tensorflow/models.git

In [ ]:
%cd /content/models/research/
!protoc object_detection/protos/*.proto --python_out=.

# Install TensorFlow Object Detection API.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

Test Model builder

In [ ]:
!python /content/models/research/object_detection/builders/model_builder_tf2_test.py

#### Persiapan file gambar dan anotasi

In [ ]:
%cd /content/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
dataset_path = '/content/drive/MyDrive/Dataset/dataset'

%mkdir images annotation
%cp -r {dataset_path}/annotation/cocacola annotation
%cp -r {dataset_path}/annotation/floridina annotation
%cp -r {dataset_path}/annotation/goodday annotation
%cp -r {dataset_path}/images/*/*.jpg images
%cp -r {dataset_path}/test .
%cp {dataset_path}/labelmap.pbtxt .

#### Memisahkan data Training 

In [ ]:
!pip install split-folders

In [ ]:
!splitfolders --output "./annotation" --ratio .8 .2 -- "./annotation" 

In [ ]:
%mkdir train val
%mv annotation/train/*/*.xml train/
%mv annotation/val/*/*.xml val/

#### Membuat file anotasi menjadi csv

In [ ]:
!wget https://raw.githubusercontent.com/nazililham11/detection_util_scripts/master/generate_csv.py

In [ ]:
!python generate_csv.py --type xml --input train --output_csv train_labels.csv 
!python generate_csv.py --type xml --input val --output_csv val_labels.csv 

#### Mengubah dataset menjadi bentuk TF Record

In [ ]:
!wget https://raw.githubusercontent.com/nazililham11/detection_util_scripts/master/generate_tfrecord.py

In [ ]:
!python generate_tfrecord.py --csv_input train_labels.csv --pbtxt_input labelmap.pbtxt --image_dir images --output_path train.record
!python generate_tfrecord.py --csv_input val_labels.csv --pbtxt_input labelmap.pbtxt --image_dir images --output_path val.record

In [ ]:
# Pindahkan file yang sudah tidak dibutuhkan
%mkdir data 
%mv -t data *.csv *.py images train val annotation

#### Persiapan pre-trained model MobileNet

In [ ]:
%cd /content
!wget http://download.tensorflow.org/models/object_detection/classification/tf2/20200710/mobilenet_v2.tar.gz
!tar -xvf mobilenet_v2.tar.gz
!rm mobilenet_v2.tar.gz

#### Persiapan file konfigurasi hyperparameter

In [ ]:
!wget https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/ssd_mobilenet_v2_320x320_coco17_tpu-8.config
!mv ssd_mobilenet_v2_320x320_coco17_tpu-8.config mobilenet_v2.config

In [ ]:
num_classes = 3
batch_size = 32
num_steps = 13000
num_eval_steps = 1000

train_record_path = '/content/train.record'
test_record_path = '/content/val.record'
model_dir = '/content/training/'
labelmap_path = '/content/labelmap.pbtxt'

pipeline_config_path = 'mobilenet_v2.config'
fine_tune_checkpoint = '/content/mobilenet_v2/mobilenet_v2.ckpt-1'

In [ ]:
import re

with open(pipeline_config_path) as f:
    config = f.read()

with open(pipeline_config_path, 'w') as f:
  # Set labelmap path
  config = re.sub('label_map_path: ".*?"', 
             'label_map_path: "{}"'.format(labelmap_path), config)
    # Set fine_tune_checkpoint path
  config = re.sub('fine_tune_checkpoint: ".*?"',
                  'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), config)
    # Set train tf-record file path
  config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 
                  'input_path: "{}"'.format(train_record_path), config)
    # Set test tf-record file path
  config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 
                  'input_path: "{}"'.format(test_record_path), config)
    # Set number of classes.
  config = re.sub('num_classes: [0-9]+',
                  'num_classes: {}'.format(num_classes), config)
    # Set batch size
  config = re.sub('batch_size: [0-9]+',
                  'batch_size: {}'.format(batch_size), config)
    # Set training steps
  config = re.sub('num_steps: [0-9]+',
                  'num_steps: {}'.format(num_steps), config)
  f.write(config)

#### Fix error saat proses training (Optional)

In [ ]:
# ImportError: cannot import name '_registerMatType' from 'cv2.cv2'
!pip install opencv-python-headless==4.1.2.30

In [ ]:
# Error DNN library is not found
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

#### Training

In [ ]:
!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_config_path} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --sample_1_of_n_eval_examples=1 \
    --num_eval_steps={num_eval_steps}

#### Validation

In [ ]:
!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_config_path} \
    --model_dir={model_dir} \
    --checkpoint_dir={model_dir}

#### Menampilkan hasil training pada Tensorboard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir '/content/training/'

#### Eksport model

In [ ]:
output_directory = 'inference_graph'

!python /content/models/research/object_detection/exporter_main_v2.py \
    --trained_checkpoint_dir {model_dir} \
    --output_directory {output_directory} \
    --pipeline_config_path {pipeline_config_path}

#### Testing model

In [ ]:
!wget https://raw.githubusercontent.com/hugozanini/object-detection/master/inferenceutils.py
from inferenceutils import *

In [ ]:
output_directory = 'inference_graph/'
test_dataset_directory = 'test/'

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(labelmap_path, use_display_name=True)
tf.keras.backend.clear_session()
model = tf.saved_model.load(f'/content/{output_directory}/saved_model')

In [ ]:
import os
images = []
for item in os.listdir(test_dataset_directory):
    if os.path.isfile(test_dataset_directory + item):
        images.append(item)
        
print(images)
print(len(images))

In [ ]:
for image_name in images:
  
    image_np = load_image_into_numpy_array(test_dataset_directory + image_name)
    output_dict = run_inference_for_single_image(model, image_np)
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        max_boxes_to_draw=200,
        min_score_thresh=.50,
        agnostic_mode=False,
        instance_masks=output_dict.get('detection_masks_reframed', None),
        use_normalized_coordinates=True,
        line_thickness=8)
    display(Image.fromarray(image_np))

#### Export ke TFJS Model

In [ ]:
!pip install tensorflowjs==2.8.0

In [ ]:
!tensorflowjs_converter \
    --input_format=tf_saved_model \
    --output_format=tfjs_graph_model \
    --signature_name=serving_default \
    --saved_model_tags=serve \
    /content/inference_graph/saved_model \
    /content/tfjs_model

In [ ]:
!zip -r /content/tfjs_model.zip tfjs_model/

Download TFJS Model

In [ ]:
from google.colab import files
files.download("/content/web_model.zip")

#### Backup / Download waight

In [ ]:
!zip -r /content/saved_model.zip /content/inference_graph/saved_model/

Download

In [ ]:
from google.colab import files
files.download("/content/saved_model.zip")

Backup ke Google Drive

In [ ]:
%cp /content/saved_model.zip /content/drive/MyDrive/Dataset/saved_model.zip

##### Restore weights

In [ ]:
%cp /content/drive/MyDrive/Dataset/saved_model.zip .
!unzip saved_model.zip -d /

### References

Blog post: https://towardsdatascience.com/custom-real-time-object-detection-in-the-browser-using-tensorflow-js-5ca90538eace

Blog post: https://gilberttanner.com/blog/tensorflow-object-detection-with-tensorflow-2-creating-a-custom-model

Notebook: https://github.com/TannerGilbert/Tensorflow-Object-Detection-API-Train-Model/blob/master/Tensorflow_2_Object_Detection_Train_model.ipynb

Blog post: https://medium.com/swlh/image-object-detection-tensorflow-2-object-detection-api-af7244d4c34e

Notebook: https://github.com/TannerGilbert/Tensorflow-Object-Detection-API-Train-Model/blob/master/Tensorflow_2_Object_Detection_Train_model.ipynb

Model Zoo: https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md

Models configs: https://github.com/tensorflow/models/tree/master/research/object_detection/configs/tf2

Object Detection API Demo : https://github.com/tensorflow/models/blob/master/research/object_detection/colab_tutorials/object_detection_tutorial.ipynb 

Inference from saved model tf2 colab: https://github.com/tensorflow/models/blob/master/research/object_detection/colab_tutorials/inference_from_saved_model_tf2_colab.ipynb

MobileNet in Browser: https://gist.github.com/woudsma/d01eeda8998c9ab972d05ec9e9843886

TFJS Converter: https://github.com/tensorflow/tfjs/tree/master/tfjs-converter
